In [32]:
from bs4 import BeautifulSoup
import requests

import random 
from time import sleep

import pandas as pd

from spellchecker import SpellChecker

## Scraping the Billbord Website for 100 Hot songs

In [75]:
#------scrapping 2021 data-------#

url = "https://www.billboard.com/charts/hot-100"
response = requests.get(url)
newest_soup = BeautifulSoup(response.content, 'html.parser')

In [76]:
response.status_code

200

In [77]:
#--------2021 song names--------#

newest_name = newest_soup.select('span.chart-element__information__song.text--truncate.color--primary')
newest_name = [t.text for t in newest_name]
#newest_name

In [78]:
#--------2021 artist names--------#

newest_artist = newest_soup.select('span.chart-element__information__artist.text--truncate.color--secondary')
newest_artist = [t.text for t in newest_artist]
#newest_artist

In [33]:
#------scrapping 2019,2020 data-------#

str = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs'
#years = [ 2006, 2007 ,2008, 2009, 2010,2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018 ,2019, 2020]
years = [ 2019, 2020]


older_song_soups = []
older_artist_soups = []

for year in years:
    url = str.format(year)
    response = requests.get(url)
    #print(response.status_code)
    
    soup = BeautifulSoup(response.content)
    older_song_soups.append(soup.find_all('div', class_ = 'ye-chart-item__title'))
    older_artist_soups.append(soup.find_all('div', class_ = 'ye-chart-item__artist'))
    
    
    wait_time = random.randint(1,4)
    sleep(wait_time)

In [35]:
#------songs & artists 2006-2020------#

for i in range(0,len(years)):
    older_song_soups[i] = [t.text.replace('\n','') for t in older_song_soups[i]]
    older_artist_soups[i] = [t.text.replace('\n','') for t in older_artist_soups[i]]
    

In [35]:
#------songs 2019------#
#older_song_soups[0] = [t.text.replace('\n','') for t in older_song_soups[0]]

#------2020------#
#older_song_soups[1] = [t.text.replace('\n','') for t in older_song_soups[1]]

#------artists 2019------#
#older_artist_soups[0] = [t.text.replace('\n','') for t in older_artist_soups[0]]

#------2020------#
#older_artist_soups[1] = [t.text.replace('\n','') for t in older_artist_soups[1]]

In [ ]:
#release_year = soup.find('span', class_ ='dropdown__year-select-date').get_text()
#song_release_year = [2006]*100 + [2007]*100 + [2008]*100 + [2009]*100 + [2010]*100 + [2011]*100 + [2012]*100 + [2013]*100 + [2014]*100 + [2015]*100 + [2016]*100 + [2017]*100 + [2018]*100 + [2019]*100 + [2020]*100 + [2021]*100

In [81]:
#----------concatinate the 3 years data together----------#

song_name = []
song_artist = []


for i in range(0,len(older_song_soups)):
        song_name += older_song_soups[i]
        song_artist += older_artist_soups[i]


song_name += newest_name
song_artist += newest_artist


In [87]:
#df = pd.DataFrame({'song_name': song_name, 'artist': song_artist, 'release_year': song_release_year})

df = pd.DataFrame({'song_name': song_name, 'artist': song_artist})
df.tail(10)

,song_name,artist
1588,Jonah,Kanye West
1589,You Should Probably Leave,Chris Stapleton
1590,Don't Go Yet,Camila Cabello
1591,Thinking 'Bout You,Dustin Lynch Featuring Lauren Alaina Or MacKen...
1592,Jesus Lord,Kanye West
1593,Summer Of Love,Shawn Mendes & Tainy
1594,Ain't Shit,Doja Cat
1595,Motley Crew,Post Malone
1596,Remote Control,Kanye West
1597,Baddest,"Yung Bleu, Chris Brown & 2 Chainz"


In [90]:
#---------Making the dataframe content lowercase-----------#

df["song_name"] = df["song_name"].apply(lambda x: x.lower())
df["artist"] = df["artist"].apply(lambda x: x.lower())

In [91]:
#-------spell checker function--------#


def input_spelling (user_input):
    spell = SpellChecker()
    corrected_output = ''
    words = spell.split_words(user_input)
    corrected_words = [spell.correction(word) for word in words]
    for i in corrected_words: 
        corrected_output = corrected_output + ' ' + i
    return corrected_output[1:]


In [92]:
input_spelling('ta to mee')

'ta to mee'

In [93]:
#--------Taking input ------------#


def user_song():
    input_song = input('Enter Your Favorite Song: ')
        
    if input_song.lower() in df.song_name.values:
        print('The recommended song for you is: ',random.choice(df['song_name'], '\n'))
        
    elif spell(input_song.lower()) in df.song_name.values:
        print('Do you mean ', spell(input_song.lower()), ' ?')
        user_song()
    
    else:
        print('Sorry, Pick another song\n')
        user_song()

In [ ]:
user_song()
